In [1]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()


/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path="../../models/hf"
model_path="../../models/hf_fin"
#model_path="../../models/hf_fin_shuffle"
#model_path="../../models/hf_10000"
model_path="../../models/hf/0312wiki300m_grad_en_to_ja_fin"
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)


/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of te

In [3]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, max_new_tokens=200, repetition_penalty=1.5)

In [12]:
import time
text_list=["今日はいい",
"富士山は",
"I have a",
"Functional polymers are"           
]

for text in text_list:
    perp=perplexity(model,tokenizer,text)
    res=pipe(text)[0]["generated_text"]
    print("-------")
    print("perplexity: ",perp)
    print("output: ",res)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
perplexity:  89.77798461914062
output:  今日はいいのに...。
 2013年4月、『SUPER GIRL』で初登場。
 2015年7月に行われた「第一回世界遺産委員会」にて、世界遺産リストに掲載された。
 2016年に開催される「第二回世界遺産会議」において、「世界遺産リストに掲載されてきた世界遺産を調査するための資格者団体」として、世界遺産リストに掲載されている。
 2018年の日本遺産協会(JAXA)では、世界遺産リストに掲載されていた世界遺産リストに掲載された世界遺産リストに掲載された世界遺産リストに掲載された世界遺産リストに掲載された世界遺産リストに掲載された世界遺産リストに


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
perplexity:  111.17222595214844
output:  富士山は、1980年(昭和56年)に開業した。

概要 
1972年に開設された「富士山」の名称で、高さ3mの大型山頂を有する山岳地帯であるが、その中には、かつて存在していた山麓部分も含まれていた。このため、現在では、山域全体としては最低峰とされている。また、山頂付近から見る場合、標識や石碑などが発掘されており、これらの遺跡は、それぞれの場所によって確認されることがある。

脚注・出典

関連項目 
 日本の山一覧
 日本百景
 富士山 (曖昧さ回避)

外部リンク 
 国土地理院地図閲覧サービス - 国土交通省関東地方整備局



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
perplexity:  258.5106506347656
output:  I have a great deal of interest."

References

External links
 Official website

1986 births
Living people The Academy Award for Best Supporting Actor is an American television series that was first broadcast on September 2, 2013 and ended October 5, 2014 with the final two-hour run from March to May 2017. It premiered at the CBS Television Network on November 18, 2013 as part of its recurring season. In this show's second season it also won three other nominations: "Outstanding Performance by a Featured Actress or Golden Globe" (with Joan
-------
perplexity:  10.568450927734375
output:  Functional polymers are involved with the reasoning of applications.

There is also an extended collection of material that can be found online at the Institute for Agricultural and Ecological Sciences, which has been published by the University of California Press since January 2019. These include:

Although it was not until December 2018 that they were 